In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
import time
from AudioStreamDescriptor import XWAVhdr
from xwav_functions import get_datetime

# allow for dynamic memory allocation
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [12]:
# now run this for a directory of xwav files

import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
import time
from AudioStreamDescriptor import XWAVhdr
from xwav_functions import get_datetime

# allow for dynamic memory allocation
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
print("GPU memory growth enabled")

# load Ann Allen model
# model = hub.load('https://www.kaggle.com/models/google/humpback-whale/TensorFlow2/humpback-whale/1')
model_path = r'C:\Users\HARP\humpback_model'
model = hub.load(model_path)
th = 0 # threshold for presence/absence of humpbacks, can play around with this

st_time = time.time()

xwavDir = 'Z:/CHNMS/CHNMS_NO_01/CHNMS_NO_01_disk01_df32' # path to xwav files
files = os.listdir(xwavDir)

for file_name in files:
        
    # build the file path
    input_path = os.path.join(xwavDir, file_name)
    print(f"Processing file: {file_name}")
    
    waveform, sample_rate = tf.audio.decode_wav(tf.io.read_file(input_path))
    
    xwav = XWAVhdr(input_path) # Marie's function to read the header info
    # Compute the number of samples in each raw file
    bits_per_byte = 8
    bytes_per_sample = xwav.xhd['NumChannels'] * \
                        xwav.xhd['BitsPerSample'] / bits_per_byte
    raw_samples = [int(b / bytes_per_sample) for b in xwav.xhd['byte_length']]
        
    # Ensure the waveform has a batch dimension and channel dimension (e.g., shape [1, time_steps, channels])
    waveform = tf.expand_dims(waveform, 0)  # Adds a batch dimension
    if len(waveform.shape) == 2:  
        waveform = tf.expand_dims(waveform, -1)  # Adds the channel dimension (shape becomes [1, time_steps, 1])
    
    # Define chunk size (number of raw files)
    num_chunks = xwav.xhd["NumOfRawFiles"]  
    
    # Function to process each chunk
    def process_chunk(chunk_waveform):
        context_step_samples = tf.cast(sample_rate, tf.int64)
        score_fn = model.signatures['score']
        scores = score_fn(waveform=chunk_waveform, context_step_samples=context_step_samples)
        predictions = scores['scores'].numpy().flatten()  # Flatten to 1D
        return predictions
    
    # List to hold the predictions
    labels = np.array([])
    starts = np.array([])
    samp = np.array([])
    spd = 60*60*24
    
    # Process each chunk
    end = 0 # start at the beginning
    for i in range(num_chunks):
        start = end + 1
        end = start + raw_samples[i]
        chunk_waveform = waveform[:, start:end, :]  # Slice the waveform in chunks
    
        # Process the chunk
        predictions = process_chunk(chunk_waveform)
        np_predictions = np.array(predictions)
        labels = np.append(labels,np_predictions)
        sec = np.arange(0,len(predictions)-1 + 1)*3.92
        st = [xwav.raw["dnumStart"][i] + timedelta(seconds=s) for s in sec]
        starts = np.append(starts,st)
        samples = np.arange(0,len(predictions)-1 + 1)*(3.92*10000) + start
        samp = np.append(samp,samples)
        # result = 1 if np.any(np_predictions > th) else 0
        # labels = np.append(labels,result)
        # all_predictions.extend(predictions)  # if you want to save all of the predictions, not just +/- for the rf
    
    df = pd.DataFrame({
        'dnumStart':starts,
        'sampleStart':samp,
        # 'dnumEnd':xwav.raw["dnumEnd"],
        'label':labels
    })

    savename = "H:/Mysticetes/Humpback_NNET/hump_nnet_no_threshold_3.92s/CHNMS_NO_01/_" + file_name[0:26] + "Humback_NNET_3.92s_window.csv"
    df.to_csv(savename,index=False)

end_time = time.time()
elapsed_time = end_time - st_time
print(f"Elasped Time: {elapsed_time:.4f} seconds")


GPU memory growth enabled
Processing file: CHNMS_NO_01_231105_154200_df32.x.wav
Processing file: CHNMS_NO_01_231106_221127_df32.x.wav
Processing file: CHNMS_NO_01_231107_202550_df32.x.wav
Processing file: CHNMS_NO_01_231108_184012_df32.x.wav
Processing file: CHNMS_NO_01_231109_165435_df32.x.wav
Processing file: CHNMS_NO_01_231110_150857_df32.x.wav
Processing file: CHNMS_NO_01_231111_132320_df32.x.wav
Processing file: CHNMS_NO_01_231112_113742_df32.x.wav
Processing file: CHNMS_NO_01_231113_095205_df32.x.wav
Processing file: CHNMS_NO_01_231114_080627_df32.x.wav
Processing file: CHNMS_NO_01_231115_062050_df32.x.wav
Processing file: CHNMS_NO_01_231116_043512_df32.x.wav
Processing file: CHNMS_NO_01_231118_010357_df32.x.wav
Processing file: CHNMS_NO_01_231118_231820_df32.x.wav
Processing file: CHNMS_NO_01_231119_213326_df32.x.wav
Processing file: CHNMS_NO_01_231120_194748_df32.x.wav
Processing file: CHNMS_NO_01_231121_180211_df32.x.wav
Processing file: CHNMS_NO_01_231122_161633_df32.x.wav
Pr

In [ ]:
# version classifying 1 raw file at a time! best option for timing, deals with noncontinuous data, gives no memory errors
# for 320 data, 1 raw file is 43.75 seconds long

import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import time
from AudioStreamDescriptor import XWAVhdr
from xwav_functions import get_datetime

# allow for dynamic memory allocation
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

st_time = time.time()

FILENAME = 'D:/Mysticetes/Humpback_NNET/CHNMS_NO_01_df32_testdata/CHNMS_NO_01_231110_150857_df32.x.wav'

waveform, sample_rate = tf.audio.decode_wav(tf.io.read_file(FILENAME))

xwav = XWAVhdr(FILENAME) # Marie's function to read the header info
# Compute the number of samples in each raw file
bits_per_byte = 8
bytes_per_sample = xwav.xhd['NumChannels'] * \
                    xwav.xhd['BitsPerSample'] / bits_per_byte
raw_samples = [int(b / bytes_per_sample) for b in xwav.xhd['byte_length']]

# test wav file
# FILENAME = 'gs://bioacoustics-www1/sounds/Cross_02_060203_071428.d20_7.wav'
FILENAME = 'D:/Mysticetes/Humpback_NNET/CHNMS_NO_01_df32_testdata/CHNMS_NO_01_231110_150857_df32.x.wav'

# load Ann Allen model
model = hub.load('https://www.kaggle.com/models/google/humpback-whale/TensorFlow2/humpback-whale/1')

# load the audio file
waveform, sample_rate = tf.audio.decode_wav(tf.io.read_file(FILENAME))

th = 0.5 # threshold for presence/absence of humpbacks

# Ensure the waveform has a batch dimension and channel dimension (e.g., shape [1, time_steps, channels])
waveform = tf.expand_dims(waveform, 0)  # Adds a batch dimension
if len(waveform.shape) == 2:  # If it's mono audio (shape is [time_steps, 1])
    waveform = tf.expand_dims(waveform, -1)  # Adds the channel dimension (shape becomes [1, time_steps, 1])

# Define chunk size (number of raw files)
num_chunks = xwav.xhd["NumOfRawFiles"]  

# Function to process each chunk
def process_chunk(chunk_waveform):
    context_step_samples = tf.cast(sample_rate, tf.int64)
    score_fn = model.signatures['score']
    scores = score_fn(waveform=chunk_waveform, context_step_samples=context_step_samples)
    predictions = scores['scores'].numpy().flatten()  # Flatten to 1D
    return predictions

# List to hold the predictions
all_predictions = []
all_st_idx = np.array([])
all_ed_idx = np.array([])
labels = np.array([])

# Process each chunk
end = 0 # start at the beginning
for i in range(num_chunks):
    start = end + 1
    end = start + raw_samples[i]
    chunk_waveform = waveform[:, start:end, :]  # Slice the waveform in chunks

    # Process the chunk
    predictions = process_chunk(chunk_waveform)
    np_predictions = np.array(predictions)
    result = 1 if np.any(np_predictions > th) else 0
    labels = np.append(labels,result)
    # all_predictions.extend(predictions)  # Append predictions from this chunk

df = pd.DataFrame({
    'dnumStart':xwav.raw["dnumStart"],
    'dnumEnd':xwav.raw["dnumEnd"],
    'label':labels
})

df.to_csv('D:/Mysticetes/Humpback_NNET/CHNMS_test_results_NEW_samples_rawFiles.csv',index=False)

end_time = time.time()
elapsed_time = end_time - st_time
print(f"Elasped Time: {elapsed_time:.4f} seconds")
